In [1]:
# Import necessary libraries

import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as bs4

Scrape Data from [Cars45](https://www.cars45.com)

In [2]:
# Get car links
base_url = "https://www.cars45.com"
urls = []

for i in tqdm(range(1, 11)):
    req = requests.get(f"{base_url}/listing?page={i}").text
    data = bs4(req, 'html.parser')
    for item in data.find_all(class_='car-feature'):
        urls.append(item['href'])

100%|█████████████████████████████████████████████| 9/9 [00:09<00:00,  1.01s/it]

['/pVXt3r6r50qW9EP9pNQ4UaVe', '/uvHXSk0p7JEbHaAte6VGmC1d', '/jOhUuxj0eqGjoNKcUXkdz259', '/6CdsLr3x5KZIyvTlFRHw8fjt', '/7S7WvZQZcIkI520aCNy42F1Q', '/3Xn2ixt0LiGZKBcMEBk2tKYS', '/BPOlZIce5EVwyq0kwZVCcFBi', '/CiMgYtvaKohZZy2DAoah0sSy', '/uH8tNfkAKk23wDmO7cizPMil', '/dXIXGOzjMhNWHd0buV9PWvgp', '/rOHjFddRoVljCSUSeJ97xwmL', '/cqoEjEv7Hj9HuVi9j6VZXzxW', '/kZWlRfwWMsRWZSPEw04UZnSV', '/x5G17vMD2GHvKeEv23P6vBKL', '/yPPwFscUX5RKGu42M4uP3fSQ', '/kmxW8aPf48lSLUKwkWQ846mA', '/dqIegH4NGMY1TV1A1OPS69jr', '/zqzzeAXLWBWcrt9xd1SNMGt4', '/CeTOrm2LTmq4wpIV1zuc3VnQ', '/zp6qxOIklsvMAUdm0gxjKx6E', '/nMT3ZIa1f2XP522bCIMvVM9b', '/gcpYFHMrC6yrv5orgdIIFeN4', '/igMByaB3FU8ZdoyCC95ZGBat', '/17DHOKAzPr2ixyehbxqzbDZA', '/wdbofMkEfXj9twLtDqvLRayx', '/3AnHmns4QUxOfaSPTSXcJ2vy', '/wtJFnGlKKpbBWAuvJjlPGxgp', '/kjwqWhNX949INI6JepeNDKew', '/axCnlgFUNe4L9fSsO0TDowlj', '/BSVfgx56quXSC7qd2lV8cAEQ', '/tUvnnwzhFAR66XaKYdAIE7a0', '/z6cqAOQwpevXvCgLsvqdqGNR', '/wCttRolR8qSSuu2boXnM5Ssp', '/6aboVzOg9CrsRVoczw9Bb9Tj', '/yyA3jOlGFqd

In [68]:
# Get car information
make = []
model = []
year = []
color = []
condition = []
mileage = []
transmission = []
price = []

for link in urls[0:10]:
    print(link)
    
    req = requests.get(f"{base_url}{link}").text
    data = bs4(req, 'html.parser')
    
    price.append(data.find("h5", class_="main-details__name__price").text.strip().replace("₦", "").lstrip())
    transmission.append(data.find("div", class_="main-details__tags flex wrap").find_all("span")[1].text.strip())
    
    description = [item.text.strip() for item in data.find_all("p", class_="general-info__name")]
    make.append(description[0])
    model.append(description[1])
    year.append(description[2])
    color.append(description[3])
    condition.append(description[4])
    mileage.append(description[5])
    

/pVXt3r6r50qW9EP9pNQ4UaVe
done
/uvHXSk0p7JEbHaAte6VGmC1d
done
/jOhUuxj0eqGjoNKcUXkdz259
done
/6CdsLr3x5KZIyvTlFRHw8fjt
done
/7S7WvZQZcIkI520aCNy42F1Q
done
/3Xn2ixt0LiGZKBcMEBk2tKYS
done
/BPOlZIce5EVwyq0kwZVCcFBi
done
/CiMgYtvaKohZZy2DAoah0sSy
done
/uH8tNfkAKk23wDmO7cizPMil
done
/dXIXGOzjMhNWHd0buV9PWvgp
done
/rOHjFddRoVljCSUSeJ97xwmL
done
11
11
11
11
11
11
11
11
